## Initialization

In [2]:
import numpy as np
import serial as ser
s = ser.Serial ('COM6', baudrate = 9600, parity = 'N', bytesize = 8, stopbits = 1, timeout = None)

In [4]:
def Rx (phi):
    t = phi * np.pi / 180
    c = np.cos (t)
    s = np.sin (t)
    return np.array ([
        [1, 0,  0, 0],
        [0, c, -s, 0],
        [0, s,  c, 0],
        [0, 0,  0, 1]
    ])

def Ry (phi):
    t = phi * np.pi / 180
    c = np.cos (t)
    s = np.sin (t)
    return np.array ([
        [c, 0,  s, 0],
        [0, 1,  0, 0],
        [-s, 0, c, 0],
        [0, 0,  0, 1]
    ])

def Rz (phi):
    t = phi * np.pi / 180
    c = np.cos (t)
    s = np.sin (t)
    return np.array ([
        [c, -s,  0, 0],
        [s, c,  0, 0],
        [0, 0, 1, 0],
        [0, 0,  0, 1]
    ])

def Tx (x):
    return np.array ([
        [1, 0, 0, x],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def Ty (y):
    return np.array ([
        [1, 0, 0, 0],
        [0, 1, 0, y],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

def Tz (z):
    return np.array ([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, z],
        [0, 0, 0, 1]
    ])

z_sec_offset = 0

def move(a):
    startpoint= np.array([90, 90, 90, 90, 90, 65])
    a_calc = startpoint + a
    command = f'{-a_calc[0]} b {a_calc[1]} s {a_calc[2]} e {a_calc[3]} v {a_calc[4]} w {a_calc[5]} g\r\n'
    res = Tx(-125) @ Rz(-a[0]) @ Tz(71.5) @ Ry(a[1]) @ Tz(125) @ Ry(a[2]) @ Tz(125) @ Ry(a[3]) @ Tz(60) @ Tz(132)
    
    if res[2,3] < z_sec_offset:
       raise Exception(f'Illegal z value: {res[2,3]} lower than {z_sec_offset}')
    else:
        s.write(command.encode())
        return res
    
def move_nocheck(a):
    startpoint= np.array([90, 90, 90, 90, 90, 65])
    a_calc = startpoint + a
    command = f'{-a_calc[0]} b {a_calc[1]} s {a_calc[2]} e {a_calc[3]} v {a_calc[4]} w {a_calc[5]} g\r\n'
    res = Tx(-125) @ Rz(-a[0]) @ Tz(71.5) @ Ry(a[1]) @ Tz(125) @ Ry(a[2]) @ Tz(125) @ Ry(a[3]) @ Tz(60) @ Tz(132)

    s.write(command.encode())
    return res

### Move to Init Position

In [5]:
move(np.array([0,0,0,0,0,0]))

array([[   1. ,    0. ,    0. , -125. ],
       [   0. ,    1. ,    0. ,    0. ],
       [   0. ,    0. ,    1. ,  513.5],
       [   0. ,    0. ,    0. ,    1. ]])

### Demo 1 (Warning: negative z / check bypass)

In [6]:
move_nocheck(np.array([0,-90,-90,-20,0,0]))

array([[  -0.93969262,    0.        ,    0.34202014, -184.33213248],
       [   0.        ,    1.        ,    0.        ,    0.        ],
       [  -0.34202014,    0.        ,   -0.93969262, -233.92098319],
       [   0.        ,    0.        ,    0.        ,    1.        ]])

In [7]:
move(np.array([0,0,0,0,0,0]))

array([[   1. ,    0. ,    0. , -125. ],
       [   0. ,    1. ,    0. ,    0. ],
       [   0. ,    0. ,    1. ,  513.5],
       [   0. ,    0. ,    0. ,    1. ]])

### Demo 2: opposite side / close to z

In [8]:
move(np.array([0,30,60,69,0,0]))

array([[ -0.93358043,   0.        ,   0.35836795, 131.30664631],
       [  0.        ,   1.        ,   0.        ,   0.        ],
       [ -0.35836795,   0.        ,  -0.93358043,   0.50573359],
       [  0.        ,   0.        ,   0.        ,   1.        ]])

### Demo 3 (Exception)

In [9]:
move(np.array([0,30,60,75,0,0]))

Exception: Illegal z value: -5.704583174446242 lower than 0

## Close Serial

In [10]:
s.close()